In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
!pip install import-ipynb
%cd /content/drive/My\ Drive/Colab\ Notebooks/RetinaSmartCamera/notebooks

/content/drive/My Drive/Colab Notebooks/RetinaSmartCamera/notebooks


In [3]:
import import_ipynb
import RetinaUtils as ru

import numpy as np
import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.utils.data.sampler import SubsetRandomSampler

from torch.utils.data.dataset import Dataset, Subset
from torch.utils.data import DataLoader
from torchvision import datasets, models, transforms

from torch.utils.tensorboard import SummaryWriter

import re
import os

importing Jupyter notebook from RetinaUtils.ipynb


In [7]:
def img_to_vector(img, R):
    batches = img.shape[0]
    x = img.shape[2]/2
    y = img.shape[1]/2
    fixation = (y,x)

    vector_batches = torch.empty(batches, 1, 50000)
    for i in batches:
        V = R.sample(img[i], fixation)
        vector_batches[i] = V

    return vector_batches

In [4]:
#Test
R = ru.Retina()

impath = "/content/drive/My Drive/Colab Notebooks/RetinaVision/botw.jpg"
img = np.float64(cv2.imread(impath, 0))
x = img.shape[1]/2
y = img.shape[0]/2
fixation = (y,x)

data_dir = '/content/drive/My Drive/Colab Notebooks/RetinaSmartCamera/retina_data/'
R.loadLoc(data_dir+'ret50k_loc.pkl')
R.loadCoeff(data_dir+'ret50k_coeff.pkl')
R.prepare(img.shape, fixation)

V = R.sample(img, fixation)

455.0


In [ ]:
#Autoencoder model from ConvAutoencoder has the following shape:
#Encoder:
#(8, 3, 257, 490) -> (8, 6, 253, 486) -> (8, 6, 126, 243) -> (8, 12, 122, 239) -> (8, 12, 61, 119) -> (8, 16, 57, 115)
#Decoder:
#(8, 12, 61, 119) -> (8, 12, 122, 239) -> (8, 6, 126, 243) -> (8, 6, 253, 486) -> (8, 3, 257, 490)

#Need to get (50000, 1) image vector to (12, 61, 119)
#(8, 1, 50000) -> 

In [9]:
class RetinaEncoder(nn.Module):
    def __init__(self, R):
        super(RetinaEncoder, self).__init__()
        self.R = R
        self.img_to_vector = img_to_vector
    
    def forward(x):
        x = self.img_to_vector(x, self.R)